In [2]:
import numpy as np 
import pandas as pd 

from keras.models import Sequential 
from keras.layers import Dense 
from keras.utils import to_categorical 
from keras.datasets import fashion_mnist 
from keras.metrics import Recall, Precision 

import matplotlib.pyplot as plt

In [3]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [4]:
train_images = (train_images / 127) - 1
test_images = (test_images / 127) - 1

In [5]:
train_images = train_images.reshape((-1, 784))
test_images = test_images.reshape((-1, 784))

In [6]:
y_t = to_categorical(train_labels)
y_tt = to_categorical(test_labels)

## Разное количество нейронов в одном слое

In [56]:
res_list = []
for i in range(0, 12):

  result = {}
  model_neurons = Sequential([
      Dense(2**i, activation='relu', input_shape=(784,)),
      Dense(10, activation='softmax')
  ])

  model_neurons.compile(
      optimizer='adam',
      loss='categorical_crossentropy',
      metrics=['accuracy']
  )

  model_neurons.fit(
      train_images,
      to_categorical(train_labels),
      epochs=5,
      batch_size=32, 
      validation_split=0.2,
      verbose=0
  )

  result_train = model_neurons.evaluate(train_images, to_categorical(train_labels), verbose=0)
  result_test = model_neurons.evaluate(test_images, to_categorical(test_labels), verbose=0)

  result['n_neurons'] = 2**i
  result['acc_train'] = np.round(result_train[1], 4)
  result['acc_test'] = np.round(result_test[1], 4)

  res_list.append(result)

In [57]:
from pprint import pprint
pprint(res_list)

[{'acc_test': 0.3602, 'acc_train': 0.3699, 'n_neurons': 1},
 {'acc_test': 0.3638, 'acc_train': 0.3672, 'n_neurons': 2},
 {'acc_test': 0.8049, 'acc_train': 0.8172, 'n_neurons': 4},
 {'acc_test': 0.8171, 'acc_train': 0.8335, 'n_neurons': 8},
 {'acc_test': 0.8436, 'acc_train': 0.8623, 'n_neurons': 16},
 {'acc_test': 0.843, 'acc_train': 0.8651, 'n_neurons': 32},
 {'acc_test': 0.8603, 'acc_train': 0.8866, 'n_neurons': 64},
 {'acc_test': 0.8686, 'acc_train': 0.8915, 'n_neurons': 128},
 {'acc_test': 0.8695, 'acc_train': 0.896, 'n_neurons': 256},
 {'acc_test': 0.8679, 'acc_train': 0.8946, 'n_neurons': 512},
 {'acc_test': 0.8743, 'acc_train': 0.9002, 'n_neurons': 1024},
 {'acc_test': 0.8677, 'acc_train': 0.8931, 'n_neurons': 2048}]


С ростом количества нейронов аккуратность сначала точность растёт, затем начинает падать. Пик аккуратности в районе от 128 до 512 нейронов. 

## Разное количество слоёв с 256 нейронов.

In [58]:
model_2l = Sequential([
      Dense(256, activation='relu', input_shape=(784,)), 
      Dense(10, activation='softmax')
  ])

model_3l = Sequential([
      Dense(256, activation='relu', input_shape=(784,)),
      Dense(256, activation='relu'),
      Dense(10, activation='softmax')
  ]) 

model_5l = Sequential([
      Dense(256, activation='relu', input_shape=(784,)),
      Dense(256, activation='relu'),
      Dense(256, activation='relu'),
      Dense(256, activation='relu'),
      Dense(10, activation='softmax')
  ]) 

model_10l = Sequential([
      Dense(256, activation='relu', input_shape=(784,)),
      Dense(256, activation='relu'),
      Dense(256, activation='relu'),
      Dense(256, activation='relu'),
      Dense(256, activation='relu'),
      Dense(256, activation='relu'),
      Dense(256, activation='relu'),
      Dense(256, activation='relu'),
      Dense(256, activation='relu'),
      Dense(10, activation='softmax')
  ])

In [59]:
res_list = []
l = [2, 3, 5, 10]
i = 0
for model in (model_2l, model_3l, model_5l, model_10l):

  result = {}

  model.summary()

  model.compile(
      optimizer='adam',
      loss='categorical_crossentropy',
      metrics=['accuracy']
  )

  model.fit(
      train_images,
      to_categorical(train_labels),
      epochs=5,
      batch_size=32, 
      validation_split=0.2,
      verbose=0
  )

  result_train = model.evaluate(train_images, to_categorical(train_labels), verbose=0)
  result_test = model.evaluate(test_images, to_categorical(test_labels), verbose=0)

  result['n_layers'] = l[i]
  i += 1
  result['acc_train'] = np.round(result_train[1], 4)
  result['acc_test'] = np.round(result_test[1], 4)

  res_list.append(result)

Model: "sequential_68"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_184 (Dense)            (None, 256)               200960    
_________________________________________________________________
dense_185 (Dense)            (None, 10)                2570      
Total params: 203,530
Trainable params: 203,530
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_69"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_186 (Dense)            (None, 256)               200960    
_________________________________________________________________
dense_187 (Dense)            (None, 256)               65792     
_________________________________________________________________
dense_188 (Dense)            (None, 10)                2570      
Total params: 269,322
Tr

In [60]:
from pprint import pprint
pprint(res_list)

[{'acc_test': 0.8663, 'acc_train': 0.8929, 'n_layers': 2},
 {'acc_test': 0.8681, 'acc_train': 0.8926, 'n_layers': 3},
 {'acc_test': 0.8644, 'acc_train': 0.8884, 'n_layers': 5},
 {'acc_test': 0.8552, 'acc_train': 0.8748, 'n_layers': 10}]


На двух слоях точность выше, чем на одном, дальше падает.

## Разное количество слоёв, примерно одно и тоже количество параметров

In [61]:
model_2l = Sequential([
      Dense(256, activation='relu', input_shape=(784,)), 
      Dense(10, activation='softmax')
  ])

model_3l = Sequential([
      Dense(222, activation='relu', input_shape=(784,)),
      Dense(222, activation='relu'),
      Dense(10, activation='softmax')
  ]) 

model_5l = Sequential([
      Dense(164, activation='relu', input_shape=(784,)),
      Dense(164, activation='relu'),
      Dense(164, activation='relu'),
      Dense(164, activation='relu'),
      Dense(10, activation='softmax')
  ]) 

model_10l = Sequential([
      Dense(128, activation='relu', input_shape=(784,)),
      Dense(128, activation='relu'),
      Dense(128, activation='relu'),
      Dense(128, activation='relu'),
      Dense(128, activation='relu'),
      Dense(128, activation='relu'),
      Dense(128, activation='relu'),
      Dense(128, activation='relu'),
      Dense(128, activation='relu'),
      Dense(10, activation='softmax')
  ])

In [62]:
res_list = []
l = [2, 3, 5, 10]
i = 0
for model in (model_2l, model_3l, model_5l, model_10l):

  result = {}

  model.summary()

  model.compile(
      optimizer='adam',
      loss='categorical_crossentropy',
      metrics=['accuracy']
  )

  model.fit(
      train_images,
      to_categorical(train_labels),
      epochs=5,
      batch_size=32, 
      validation_split=0.2,
      verbose=0
  )

  result_train = model.evaluate(train_images, to_categorical(train_labels), verbose=0)
  result_test = model.evaluate(test_images, to_categorical(test_labels), verbose=0)

  result['n_layers'] = l[i]
  i += 1
  result['acc_train'] = np.round(result_train[1], 4)
  result['acc_test'] = np.round(result_test[1], 4)

  res_list.append(result)

Model: "sequential_72"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_204 (Dense)            (None, 256)               200960    
_________________________________________________________________
dense_205 (Dense)            (None, 10)                2570      
Total params: 203,530
Trainable params: 203,530
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_73"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_206 (Dense)            (None, 222)               174270    
_________________________________________________________________
dense_207 (Dense)            (None, 222)               49506     
_________________________________________________________________
dense_208 (Dense)            (None, 10)                2230      
Total params: 226,006
Tr

In [63]:
from pprint import pprint
pprint(res_list)

[{'acc_test': 0.8651, 'acc_train': 0.889, 'n_layers': 2},
 {'acc_test': 0.8684, 'acc_train': 0.8932, 'n_layers': 3},
 {'acc_test': 0.8652, 'acc_train': 0.8891, 'n_layers': 5},
 {'acc_test': 0.8361, 'acc_train': 0.8616, 'n_layers': 10}]


При одинаковои числе параметров точность быстро падает. 

## Тестируем разные оптимизаторы

In [64]:
res_list = []

for opt in ['SGD', 'Adam', 'RMSProp']:

  result = {}

  model_opti = Sequential([
      Dense(256, activation='relu', input_shape=(784,)), 
      Dense(10, activation='softmax')
  ])

  model_opti.compile(
      optimizer=opt,
      loss='categorical_crossentropy',
      metrics=['accuracy']
  )

  model_opti.fit(
      train_images,
      to_categorical(train_labels),
      epochs=5,
      batch_size=32, 
      validation_split=0.2,
      verbose=0
  )

  result_train = model_opti.evaluate(train_images, to_categorical(train_labels), verbose=0)
  result_test = model_opti.evaluate(test_images, to_categorical(test_labels), verbose=0)

  result['opt'] = opt
  i += 1
  result['acc_train'] = np.round(result_train[1], 4)
  result['acc_test'] = np.round(result_test[1], 4)

  res_list.append(result)

In [65]:
from pprint import pprint
pprint(res_list)

[{'acc_test': 0.8554, 'acc_train': 0.8741, 'opt': 'SGD'},
 {'acc_test': 0.8607, 'acc_train': 0.8902, 'opt': 'Adam'},
 {'acc_test': 0.8677, 'acc_train': 0.8971, 'opt': 'RMSProp'}]


Adam справляет чуть лучше RMSProp

## Выводы 

На кчество влияет оптимальное число слоёв и нейронов. Функции активации тоже, но её я не тестировал, это и так очевидно. 

## Классы

In [51]:
model_opti = Sequential([
      Dense(256, activation='relu', input_shape=(784,)), 
      Dense(10, activation='softmax')
  ])

model_opti.compile(
      optimizer=opt,
      loss='categorical_crossentropy',
      metrics=['accuracy']
  )

model_opti.fit(
      train_images,
      to_categorical(train_labels),
      epochs=5,
      batch_size=32, 
      validation_split=0.2,
      verbose=0
  )

In [53]:
predictions = model_2l.predict(test_images)

# Print our model's predictions.
test_labels_predict = np.argmax(predictions, axis=1)

In [55]:
labels = np.unique(train_labels)
df = pd.DataFrame(columns=['recall', 'precision'], index=range(10))

for i in labels:
  recall = Recall()
  recall.update_state(y_true=(test_labels == i), 
                      y_pred=(test_labels_predict == i))
  df.loc[i, 'recall'] = recall.result().numpy()

  precision = Precision()
  precision.update_state(y_true=(test_labels == i), 
                         y_pred=(test_labels_predict == i))
  df.loc[i, 'precision'] = precision.result().numpy()

  df.loc[i, 'F'] = 2*recall.result().numpy()*precision.result().numpy()/(precision.result().numpy()+recall.result().numpy())

df.index = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

df

,recall,precision,F
T-shirt/top,0.807,0.838006,0.822211
Trouser,0.976,0.935762,0.955458
Pullover,0.807,0.741728,0.772988
Dress,0.865,0.866733,0.865866
Coat,0.833,0.778505,0.804831
Sandal,0.93,0.963731,0.946565
Shirt,0.62,0.740741,0.675014
Sneaker,0.898,0.943277,0.920082
Bag,0.97,0.965174,0.967581
Ankle boot,0.971,0.899907,0.934103
